In [29]:
import numpy as np                                      #import required libraries 
from matplotlib import *
from math import *
from sympy import *
import matplotlib.pyplot as plt

In [30]:
class Nacaxxxx_wing:                                           # class for NACA airfoil to get x and y coordinates of airfoil
    
    def __init__(self, digits, c, b):
        self.digits = str(digits)
        self.c = c  
        self.b = b                                                               # replace x by x/c for scaling to the chord length
        self.m = float(self.digits[0])*(.01)*c                                       # max camber, m = (first digit from left)* (c/100)
        self.p = float(self.digits[1])*(.1)*c                                        # pos_max camber, p = (2nd digit from left)* (c/10)
        self.tmax = float(self.digits[2:])*(.01)*c                                   # max thickness, tmax = (last two digit)* (c/100)
        

    def camber(self, x):
        m = self.m
        p = self.p
        c = self.c

        if p == 0:
            yc = m*(c-x)*(1 + x/c - 2*p)/((1-p)**2)
        
        else:

            if 0 < x <= p*c :
                yc = (m/(p*p))*(2*p*x - x*x/c)
                
            else:
                yc = m*(c-x)*(1 + x/c - 2*p)/((1-p)**2)
            
        #print("camber z coordinates: ", yc)
        return yc
            
    def thickness(self, x):
        tmax = self.tmax
        c = self.c
        #yt = ((tmax*c)/0.2)*(0.2969*sqrt(x/c) - 0.1260*(x/c) - 0.3516*pow(x/c, 2) + 0.2843*pow(x/c, 3) - 0.1015*pow(x/c, 4))
        yt = (5*tmax*c)*(0.2969*(x/c)**0.5-0.126*(x/c)-0.3516*(x/c)**2+0.2843*(x/c)**3-0.1036*(x/c)**4)
        #print("thickness-camber to boundary length: ", yt)
        return yt
        
    def ftheta(self, x):
        m = self.m
        p = self.p
        c = self.c

        if p == 0:
            if x < 1e-6:
                theta1 = 0
            else:
                z = symbols('z')
                yc = m*(c-z)*(1 + z/c - 2*p)/((1-p*p)**2)
                yc_ = lambdify(z, yc, "numpy")
                value = yc_(x)
                #yc_ = diff(yc, z)
                #value = yc_.subs(z,x)
                theta1 = np.arctan2(value, 1)

        else:
            if 0 <= x <= p:
                z = symbols('z')
                yc = (m*z/(p*p))*(2*p - z/c)
                #yc_ = diff(yc, z)
                yc_ = lambdify(z, yc, "numpy")
                value = yc_(x)
                #value = yc_.subs(z,x)
                theta1 = np.arctan2(value, 1)

            elif p < x <= c:
                z = symbols('z')
                yc = m*(c-z)*(1 + z/c - 2*p)/((1-p*p)**2)
                yc_ = lambdify(z, yc, "numpy")
                value = yc_(x)
                #yc_ = diff(yc, z)
                #value = yc_.subs(z,x)
                theta1 = np.arctan2(value, 1)
        
        #print("theta1 is: ", theta1)
        return theta1      

    
    def get_Xu(self, x):
        self.Xu = x - self.thickness(x) * np.sin(self.ftheta(x))
        return self.Xu
    
    def get_Zu(self, x):
        #self.Zu = self.camber(x) + self.thickness(x) * np.cos(self.ftheta(x))
        self.Zu = self.camber(x) + self.thickness(x)
        #print("value of Zu is: ", self.Zu)
        return self.Zu
    
    def get_Xl(self, x):
        self.Xl = x + self.thickness(x) * np.sin(self.ftheta(x))
        return self.Xl
    
    def get_Zl(self, x):
        #self.Zl = self.camber(x) - self.thickness(x) * np.cos(self.ftheta(x))
        self.Zl = self.camber(x) - self.thickness(x) 
        #print("value of Zl is: ", self.Zl)
        return self.Zl



In [31]:
class vortex_lattice_method:
    
    def __init__(self, n, c, b, U_free, alpha, Yc, del_camber):           #Yc = will take self.camber as parameter while calling the vortex_lattice_method class
        self.n = n
        self.c = c
        self.b = b
        self.U_free = U_free
        self.alpha = alpha
        self.Yc = Yc
        self.del_camber = del_camber


        self.X_1n = np.zeros(2*n)
        self.X_2n = np.zeros(2*n)                              
        self.Y_1n = np.zeros(2*n)
        self.Y_2n = np.zeros(2*n)
        self.Z_1n = np.zeros(2*n)
        self.Z_2n = np.zeros(2*n)
        self.bound_vortex_coordinates()

        self.X_control = np.zeros(2*n)
        self.Y_control = np.zeros(2*n)
        self.Z_control = np.zeros(2*n)
        self.control_coordinates()

        self.u_control_j_vortex = np.zeros(2*n)
        self.v_control_j_vortex = np.zeros(2*n)
        self.w_control_j_vortex = np.zeros(2*n)
        self.u_vel_control_pt = np.zeros((2*n,2*n))
        self.v_vel_control_pt = np.zeros((2*n,2*n))
        self.w_vel_control_pt = np.zeros((2*n,2*n))
        self.coeff_vel_control_pt()

        self.matrix_B = np.zeros((n,2*n))
        self.matrix_U = np.zeros(n)
        self.matrix_T = np.zeros(n)
        self.fsolve_T()

        


    def bound_vortex_coordinates(self):
        n = self.n
        c = self.c
        b = self.b
        X_1n = self.X_1n
        X_2n = self.X_2n
        Y_1n = self.Y_1n
        Y_2n = self.Y_2n
        Z_1n = self.Z_1n
        Z_2n = self.Z_2n
        Yc = self.Yc

        j = -n
        for i in range(0, 2*n):
            X_1n[i] = c/4
            X_2n[i] = c/4
            Y_1n[i] = (j*b)/n
            Y_2n[i] = (j+1)*b/n
            Z_1n[i] = Yc(c/4)
            Z_2n[i] = Yc(c/4)
            j = j + 1
        
        self.X_1n = X_1n
        self.X_2n = X_2n
        self.Y_1n = Y_1n
        self.Y_2n = Y_2n
        self.Z_1n = Z_1n
        self.Z_2n = Z_2n

        print("Y_1n", Y_1n)
        print("Y_2n", Y_2n)
        

    def control_coordinates(self):
        n = self.n
        c = self.c
        b = self.b
        X_control = self.X_control
        Y_control = self.Y_control
        Z_control = self.Z_control
        Y_1n = self.Y_1n
        Y_2n = self.Y_2n
        Yc = self.Yc

        for i in range(0, 2*n):
            X_control[i] = 3*c/4
            Y_control[i] = (Y_1n[i] + Y_2n[i])/2
            Z_control[i] = Yc(3*c/4)

        self.X_control = X_control
        self.Y_control = Y_control
        self.Z_control = Z_control

        print("Y_control", Y_control)
    
    def coeff_vel_AB(self, j, x, y, z):                 #coeff_velocity induced at (x,y,z) due to j_th bound vortex
        X_1n = self.X_1n
        X_2n = self.X_2n
        Y_1n = self.Y_1n
        Y_2n = self.Y_2n
        Z_1n = self.Z_1n
        Z_2n = self.Z_2n

        r0 = [(X_2n[j]-X_1n[j]), (Y_2n[j]-Y_1n[j]), (Z_2n[j]-Z_1n[j])]           #list
        r1 = [(x-X_1n[j]), (y-Y_1n[j]), (z-Z_1n[j])]
        r2 = [(x-X_2n[j]), (y-Y_2n[j]), (z-Z_2n[j])]

        r_0 = np.array(r0)                                                       #array
        r_1 = np.array(r1)
        r_2 = np.array(r2)

        cross_r1_r2 = np.cross(r_1, r_2)
        fac1_AB = ((cross_r1_r2)/(np.linalg.norm(cross_r1_r2))**2)                #np.linalg.norm calculates modulus of a vector

        unit_r1 = r_1/(np.linalg.norm(r_1))
        unit_r2 = r_2/(np.linalg.norm(r_2))
        fac2_AB = (np.dot(r_0, unit_r1) - np.dot(r_0, unit_r2))

        uvw_vel_AB = (fac1_AB)*(fac2_AB)
        print("uvw_vel_AB", uvw_vel_AB)

        return (uvw_vel_AB)

    def coeff_vel_A0(self, j, x, y, z ):
        X_1n = self.X_1n
        X_2n = self.X_2n
        Y_1n = self.Y_1n
        Y_2n = self.Y_2n
        Z_1n = self.Z_1n
        Z_2n = self.Z_2n

        den_fac1_A0 = ((z-Z_1n[j])**2 + (Y_1n[j]-y)**2)
        fac1A0 = [0, ((z-Z_1n[j])/den_fac1_A0), ((Y_1n[j]-y)/den_fac1_A0)]    #list
        fac1_A0 = np.array(fac1A0)      #array

        fac2_A0 = (1+((x-X_1n[j])/(sqrt((x-X_1n[j])**2 + (y-Y_1n[j])**2 + (y-Y_1n[j])**2 ))))

        uvw_vel_A0 = (fac1_A0)*(fac2_A0)
        print("uvw_vel_A0 is ", uvw_vel_A0)

        return (uvw_vel_A0)
    
    def coeff_vel_B0(self, j, x, y, z ):
        X_1n = self.X_1n
        X_2n = self.X_2n
        Y_1n = self.Y_1n
        Y_2n = self.Y_2n
        Z_1n = self.Z_1n
        Z_2n = self.Z_2n

        den_fac1_B0 = ((z-Z_2n[j])**2 + (Y_2n[j]-y)**2)
        fac1B0 = [0, (z-Z_2n[j])/den_fac1_B0, (Y_2n[j]-y)/den_fac1_B0]        #list
        fac1_B0 = np.array(fac1B0)       #array

        fac2_B0 = (1+((x-X_2n[j])/(sqrt((x-X_2n[j])**2 + (y-Y_2n[j])**2 + (y-Y_2n[j])**2 ))))    
        
        uvw_vel_B0 = (fac1_B0)*(fac2_B0)
        print("uvw_vel_B0 is ", uvw_vel_B0)

        return (uvw_vel_B0)

    
    def test_AB(self):
        u_AB = self.coeff_vel_AB(1, 1, 1, 1)
        print("u_AB[1] is :", u_AB[1])

    def test_A0(self):
        u_A0 = self.coeff_vel_A0(1, 1, 1, 1)
        print("u_A0[1] is :", u_A0[1])

    def test_B0(self):
        u_B0 = self.coeff_vel_B0(1, 1, 1, 1)
        print("u_B0[1] is :", u_B0[1])

    def coeff_vel_control_pt(self):     #coeff of total induced velocity at m_th control point due to all n vortex systems 
        X_control = self.X_control
        Y_control = self.Y_control
        Z_control = self.Z_control
        n = self.n
        u_control_j_vortex = self.u_control_j_vortex
        v_control_j_vortex = self.v_control_j_vortex
        w_control_j_vortex = self.w_control_j_vortex
        u_vel_control_pt = self.u_vel_control_pt
        v_vel_control_pt = self.v_vel_control_pt
        w_vel_control_pt = self.w_vel_control_pt

        for m in range(0, 2*n):
            for j in range(0, 2*n):                                        #due to j_th panel
                vel_AB = self.coeff_vel_AB(j, X_control[m], Y_control[m], Z_control[m])
                vel_A0 = self.coeff_vel_A0(j, X_control[m], Y_control[m], Z_control[m])
                vel_B0 = self.coeff_vel_B0(j, X_control[m], Y_control[m], Z_control[m])

                u_control_j_vortex[j] = vel_AB[0] + vel_A0[0] + vel_B0[0]    #array having coeff_x_component of velocity induced at m_th control point due to j_th panel
                v_control_j_vortex[j] = vel_AB[1] + vel_A0[1] + vel_B0[1]
                w_control_j_vortex[j] = vel_AB[2] + vel_A0[2] + vel_B0[2]
        
                u_vel_control_pt[m][j] = u_control_j_vortex[j]                  #coeff_x_component of total velocity induced at m_th control point due to all the vortex systems(all panels)
                v_vel_control_pt[m][j] = v_control_j_vortex[j]
                w_vel_control_pt[m][j] = w_control_j_vortex[j]

        self.u_vel_control_pt = u_vel_control_pt            
        self.v_vel_control_pt = v_vel_control_pt
        self.w_vel_control_pt = w_vel_control_pt

        print("u_vel_control_pt is ", u_vel_control_pt)
        print("v_vel_control_pt is ", v_vel_control_pt)
        print("w_vel_control_pt is ", w_vel_control_pt)

    
    def fsolve_T(self):
        n = self.n
        c = self.c
        U_free = self.U_free
        alpha = self.alpha
        X_control = self.X_control
        del_camber = self.del_camber

        matrix_B = self.matrix_B
        matrix_T = self.matrix_T               #circulation matrix
        matrix_U = self.matrix_U                  
        
        u_vel_control_pt = self.u_vel_control_pt 
        v_vel_control_pt = self.v_vel_control_pt
        w_vel_control_pt = self.w_vel_control_pt
        
        
        for m in range(n):
            for j in range(2*n):
                matrix_B[m][j] = (1/(4*np.pi))*(-(u_vel_control_pt[m][j])*np.sin(del_camber(X_control[m])) + (w_vel_control_pt[m][j])*np.cos(del_camber(X_control[m])))
        
        matrix_C = np.zeros((n,n))
        for i in range(n):
            for j in range(n):
                matrix_C[i][j]= matrix_B[i][j] + matrix_B[i][n+j]

        print("matrix_C is ", matrix_C)

        for i in range(n):
            matrix_U[i] = -(U_free*(np.sin(alpha - del_camber(X_control[i]))))

        print("matrix_U is ", matrix_U)



        matrix_T = np.linalg.solve(matrix_C, matrix_U)
        print("matrix_T is", matrix_T)

        wing_matrix_T = np.zeros(2*n)
        for i in range(n):
            wing_matrix_T[i] = matrix_T[i]
            wing_matrix_T[n+i] = matrix_T[n-i-1]
        print("wing_matrix_T is", wing_matrix_T)  


        self.matrix_T = matrix_T
        self.wing_matrix_T = wing_matrix_T


    
    def get_Cl(self):                     #sectional lift coefficient
        c = self.c
        n = self.n
        wing_matrix_T = self.wing_matrix_T
        U_free = self.U_free
    
        sect_Cl = np.zeros(2*n)
        for i in range(2*n):
            sect_Cl[i] = 2*(wing_matrix_T[i])/(U_free*c)
        
        self.sect_Cl = sect_Cl
        print("sect_Cl is", sect_Cl)
    
    #def get_Cp(self):



    

 for lift coefficient : For wings that have no dihedral over any portion of the wing, all the lift is generated by the free-stream velocity crossing the spanwise vortex filament, since there are no sidewash or backwash velocities. Furthermore, since the panels extend from the leading edge to the trailing edge, the lift acting on the n th panel is:
ln = rUn

In [32]:

digits = (input("enter the four digits of NACAXXXX airfoil: "))
#digits = 2124
c = float(input("enter the chord length: "))
#c = 1
b = float(input("enter the span of rectangular wing: "))
#b = 6
n = int(input("number of spanwise panels"))
#n = 20
U_free = float(input("enter the free stream velocity in m/s: "))
#U_free = 70
alpha = float(input("enter angle of attack of wing in degree: "))*np.pi/180
#alpha = np.pi/18


wing_1 = Nacaxxxx_wing(digits, c, b)
result_vlm1 = vortex_lattice_method(n, c, b, U_free, alpha, wing_1.camber, wing_1.ftheta)
result_vlm1.bound_vortex_coordinates()
result_vlm1.control_coordinates()
result_vlm1.coeff_vel_AB(1, 1, 1, 1)
result_vlm1.test_AB()
result_vlm1.test_A0()
result_vlm1.test_B0()
result_vlm1.get_Cl()

Y_1n [-6.  -5.7 -5.4 -5.1 -4.8 -4.5 -4.2 -3.9 -3.6 -3.3 -3.  -2.7 -2.4 -2.1
 -1.8 -1.5 -1.2 -0.9 -0.6 -0.3  0.   0.3  0.6  0.9  1.2  1.5  1.8  2.1
  2.4  2.7  3.   3.3  3.6  3.9  4.2  4.5  4.8  5.1  5.4  5.7]
Y_2n [-5.7 -5.4 -5.1 -4.8 -4.5 -4.2 -3.9 -3.6 -3.3 -3.  -2.7 -2.4 -2.1 -1.8
 -1.5 -1.2 -0.9 -0.6 -0.3  0.   0.3  0.6  0.9  1.2  1.5  1.8  2.1  2.4
  2.7  3.   3.3  3.6  3.9  4.2  4.5  4.8  5.1  5.4  5.7  6. ]
Y_control [-5.85 -5.55 -5.25 -4.95 -4.65 -4.35 -4.05 -3.75 -3.45 -3.15 -2.85 -2.55
 -2.25 -1.95 -1.65 -1.35 -1.05 -0.75 -0.45 -0.15  0.15  0.45  0.75  1.05
  1.35  1.65  1.95  2.25  2.55  2.85  3.15  3.45  3.75  4.05  4.35  4.65
  4.95  5.25  5.55  5.85]
uvw_vel_AB [-0.02269111  0.         -1.14873766]
uvw_vel_A0 is  [0 -0.839411417181086 -12.7485608984378]
uvw_vel_B0 is  [0 -0.839411417181096 12.7485608984379]
uvw_vel_AB [-0.01506952  0.         -0.76289467]
uvw_vel_A0 is  [0 -0.839411417181096 12.7485608984379]
uvw_vel_B0 is  [0 -0.0788671545608468 3.59338472967857]
uvw_vel